# PROYECTO 2. RKG (RESTART KERNEL GIRLS)
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import re
import sys

sys.path.append('../')
from src import soporte_variables as spv
from src import soporte_funciones as spf

pd.options.display.max_columns = None

## TRATAMIENTO Y LIMPIEZA DE LOS DATOS A ESTUDIAR

## 1. Unión de las preguntas separadas en varias columnas (q10, q39)

In [2]:
# abrimos el .csv con los datos de Data Analyst para las preguntas relevantes seleccionadas
df = pd.read_csv('../resultados/df_da_relevante.csv', index_col = 0)
df = df.reset_index()
df.sample(2)

,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41
43,587,35-39,Man,Kenya,Bachelor’s degree,1-3 years,"Python, R, SQL",Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc)",Kaggle Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1745,19470,25-29,Man,United States of America,Master’s degree,1-3 years,"Python, R, SQL",SQL,"RStudio ,Jupyter Notebook",NaN,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,"10,000 or more employees",5-9,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Business intelligence software (Salesforce, Ta..."


In [3]:
# sacamos los nulos para monitorearlos a lo largo del proceso
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
index,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41
0,0,0,0,0,0,0,143,144,146,1508,1639,2191,2287,2124,2286,2204,2250,2267,2132,2114,2231,2281,2280,2276,1643,2206,193,114,130,150,227,1062,1069,1858,2266,2252,2278,1739,2208,1863,2104,2251,1884,2253,533


In [4]:
# creamos una lista con las respuestas de la q10 unificadas
lista_futura_q10 = []

for indice, row in df.iterrows():
    #print(indice, row.index)
    valor = ""
    for i in row.index:
        if "q10" in i:
            if type(row[i]) != float:
                row[i] = row[i].strip()
                valor += row[i] + ","
                if valor == ",":
                    valor = ''
    if valor == '':
        valor = np.nan
    try:
        lista_futura_q10.append(valor.strip(','))
    except:
        lista_futura_q10.append(valor)
lista_futura_q10[:5]

['Kaggle Notebooks,Colab Notebooks,Databricks Collaborative Notebooks',
 'Google Cloud Datalab',
 'Kaggle Notebooks,Colab Notebooks',
 'Kaggle Notebooks,Azure Notebooks',
 'IBM Watson Studio']

In [5]:
# creamos una lista con las respuestas de la q39 unificadas
lista_futura_q39 = []

for indice, row in df.iterrows():
    #print(indice, row.index)
    valor = ""
    for i in row.index:
        if "q39" in i:
            if type(row[i]) != float:
                row[i] = row[i].strip()
                valor += row[i] + ","
                if valor == ",":
                    valor = ''
    if valor == '':
        valor = np.nan
    try:
        lista_futura_q39.append(valor.strip(','))
    except:
        lista_futura_q39.append(valor)
lista_futura_q39[:5]

[nan,
 nan,
 'I do not share my work publicly',
 nan,
 'I do not share my work publicly']

In [6]:
# creamos las columnas unificadas en el dataframe y borramos las sobrantes
df['q10'] = lista_futura_q10
df['q39'] = lista_futura_q39
df.head(3)

,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41,q10,q39
0,16,50-54,Man,Belgium,Bachelor’s degree,20+ years,"Python, SQL",Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"1000-9,999 employees",5-9,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Local development environments (RStudio, Jupyt...","Kaggle Notebooks,Colab Notebooks,Databricks Co...",NaN
1,32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,< 1 years,R,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,250-999 employees,3-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN
2,33,30-34,Woman,Egypt,Bachelor’s degree,3-5 years,Python,R,"Notepad++,Jupyter Notebook",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Non-profit/Service,0-49 employees,0,Analyze and understand data to influence produ...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,"Basic statistical software (Microsoft Excel, G...","Kaggle Notebooks,Colab Notebooks",I do not share my work publicly


In [7]:
df.drop(columns=spv.columnas_sobrantes, inplace=True)
df.sample()

,index,age,gender,q3,q4,q6,q7,q8,q9,q14,q20,q21,q22,q24,q32,q34,q35,q41,q10,q39
1112,12456,25-29,Man,Other,Bachelor’s degree,5-10 years,"Python, R, SQL, MATLAB, Other",Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...","Matplotlib ,Seaborn ,Plotly / Plotly Express ...",Hospitality/Entertainment/Sports,50-249 employees,1-2,Analyze and understand data to influence produ...,"PostgreSQL , Oracle Database , Amazon RDS , A...","Looker, Tableau, Tableau CRM",Tableau,NaN,"Kaggle Notebooks,Google Cloud Notebooks (AI Pl...",NaN


In [8]:
# reordenamos las columnas
df = df.reindex(columns = ['age', 'gender', 'q3', 'q4', 'q6', 'q7', 'q8', 'q9', 'q10', 'q14', 'q20', 'q21', 'q22', 'q24', 'q32', 'q34', 'q35', 'q39', 'q41'])

In [9]:
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
index,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
0,0,0,0,0,0,143,144,146,151,193,114,130,150,227,1062,1069,1858,1099,533


In [10]:
df.sample(3)

,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
1105,40-44,Man,Ireland,Master’s degree,1-3 years,"Python, SQL",SQL,"Visual Studio Code (VSCode) , Notepad++, Subli...",None,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Manufacturing/Fabrication,"1000-9,999 employees",3-4,Analyze and understand data to influence produ...,"PostgreSQL , Oracle Database , Amazon Redshift","Tableau, Salesforce, Other",Other,I do not share my work publicly,"Business intelligence software (Salesforce, Ta..."
1286,30-34,Man,Japan,Master’s degree,< 1 years,"C, Bash",Python,"Notepad++,Jupyter Notebook",Colab Notebooks,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Manufacturing/Fabrication,"1000-9,999 employees",5-9,Analyze and understand data to influence produ...,None,None,NaN,GitHub,Other
263,40-44,Woman,Egypt,Professional doctorate,10-20 years,"Python, SQL, C, C++",Python,"Visual Studio ,PyCharm , Sublime Text,Jupyter ...",Colab Notebooks,"Matplotlib ,Seaborn",Government/Public Service,0-49 employees,5-9,None of these activities are an important par...,None,None,NaN,GitHub,"Basic statistical software (Microsoft Excel, G..."


## 2. Tratamiento de las columnas que separaremos con .explode() más adelante

In [11]:
# limpiamos la columna 'q9'
df['q9'] = df['q9'].apply(spf.limpiar_q9)
df['q9'].reset_index().sample(10)

,index,q9
1044,1044,"Jupyter-JupyterLab-JupyterNotebooks ,RStudio ,..."
2023,2023,RStudio
1691,1691,"RStudio ,Visual Studio ,PyCharm , Spyder, Note..."
855,855,Sublime Text
2047,2047,"Visual Studio Code (VSCode) ,Jupyter Notebook"
1523,1523,"Visual Studio ,PyCharm ,Jupyter Notebook, Other"
298,298,RStudio
206,206,"RStudio ,Jupyter Notebook"
1286,1286,"Notepad++,Jupyter Notebook"
2140,2140,"Spyder,Jupyter Notebook"


In [12]:
# limpiamos la columna 'q24'
df['q24'] = df['q24'].apply(spf.limpiar_q24)
df['q24'].reset_index().sample(10)

,index,q24
2124,2124,Analyze and understand data to influence produ...
670,670,Analyze and understand data to influence produ...
2016,2016,Experimentation and iteration to improve exis...
1707,1707,None of these activities are an important par...
1182,1182,NaN
2095,2095,Analyze and understand data to influence produ...
1735,1735,Analyze and understand data to influence produ...
2146,2146,Analyze and understand data to influence produ...
205,205,NaN
1900,1900,Analyze and understand data to influence produ...


In [13]:
# limpiamos la columna 'q41'
df['q41'] = df['q41'].apply(spf.limpiar_q41a)
df['q41'] = df['q41'].apply(spf.limpiar_q41b)
df['q41'] = df['q41'].apply(spf.limpiar_q41c)
df['q41'] = df['q41'].apply(spf.limpiar_q41d)
df['q41'] = df['q41'].apply(spf.limpiar_q41e)
df['q24'].reset_index().sample(10)

,index,q24
2103,2103,Analyze and understand data to influence produ...
2169,2169,Build prototypes to explore applying machine ...
1350,1350,Analyze and understand data to influence produ...
1041,1041,Analyze and understand data to influence produ...
2180,2180,None of these activities are an important par...
66,66,Analyze and understand data to influence produ...
1595,1595,Analyze and understand data to influence produ...
165,165,Analyze and understand data to influence produ...
2254,2254,Analyze and understand data to influence produ...
2101,2101,Analyze and understand data to influence produ...


In [14]:
# hacemos un split en las columnas en las que más adelante haremos un .explode()
for col in spv.columnas_unidas:
    df[col] = df[col].str.split(',')

In [15]:
#chequeamos los nulos
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
index,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
0,0,0,0,0,0,143,144,146,151,193,114,130,150,227,1062,1069,1858,1099,533


## 3. Creación de una nueva columna para agrupar los paises por continentes

In [16]:
# creamos un diccionario que asocie cada país con su continente correspondiente
# con él creamos una nueva columna con los continentes correspondientes a cada país
df['continente'] = df['q3'].map(spv.continentes)

In [17]:
df.sample(3)

,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41,continente
639,22-24,Man,India,Master’s degree,1-3 years,"[Python, SQL]",Python,"[PyCharm , Spyder, Notepad++, Jupyter Notebook]","[Kaggle Notebooks, Colab Notebooks, Binder / J...","[ Matplotlib , Seaborn ]",[Computers/Technology],"10,000 or more employees",10-14,[Analyze and understand data to influence prod...,"[MySQL , Oracle Database , MongoDB ]","[ Microsoft Power BI, Tableau]",Tableau,"[Kaggle, Colab]",[Basic statistical software (Microsoft Excel-G...,Asia
1410,22-24,Man,India,Master’s degree,< 1 years,"[Python, SQL, Java]",Python,"[Jupyter-JupyterLab-JupyterNotebooks , RStudio...","[Colab Notebooks, Binder / JupyterHub]","[ Matplotlib , Seaborn , Plotly / Plotly Expre...",[Retail/Sales],"10,000 or more employees",3-4,[ Experimentation and iteration to improve exi...,[ None],"[ Tableau, Tableau CRM]",Tableau,[I do not share my work publicly],[Basic statistical software (Microsoft Excel-G...,Asia
548,22-24,Man,India,Bachelor’s degree,< 1 years,[Python],Python,"[Visual Studio Code (VSCode) , Jupyter Notebook]","[Kaggle Notebooks, Colab Notebooks]","[ Matplotlib , Seaborn ]",[Online Service/Internet-based Services],0-49 employees,1-2,[ Do research that advances the state of the a...,NaN,NaN,NaN,NaN,[Basic statistical software (Microsoft Excel-G...,Asia


## 4. Guardado de los datos

In [18]:
df.to_csv('../resultados/df_da_relevante_limpio.csv')
df.to_pickle('../resultados/df_da_relevante_limpio.pkl')